# Testing Expasy chat

These tests requires to start the expasy chat stack locally with `docker compose -f compose.dev.yml up`

Testing our Expasy query helper reusing the example queries that succeeded retrieved from `test_example_queries.ipynb`

Potential interesting questions to use for evaluation:
- Which are the human genes associated with cancer and their orthologs expressed in the rat brain?
- Select the number of proteins for each of the subclasses of ec:1.1.1.- EC (Enzyme Commission) class (from biosoda)

In [1]:
import os
from collections import defaultdict

import pandas as pd
import requests
from dotenv import load_dotenv

from expasy_chat.config import get_llm_client, settings
from expasy_chat.utils import query_sparql
from expasy_chat.validate_sparql import extract_sparql_queries

load_dotenv()
expasy_api_key = os.getenv("EXPASY_API_KEY")

example_queries = [
    {
        "question": "What is the accession number in uniprot of the human gene LCT? Return only unique protein URIs",
        "endpoint": "https://sparql.uniprot.org/sparql/",
        "query": """PREFIX up: <http://purl.uniprot.org/core/>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
    SELECT DISTINCT ?protein
    WHERE{
        ?protein a up:Protein .
        ?protein up:organism taxon:9606 .
        ?protein up:encodedBy ?gene .
        ?gene skos:prefLabel "LCT" .
    }""",
    },
    {
        # NOTE: The "mature" part in the question makes it harder to answer
        # "question": "How do I filter for reviewed (mouse) proteins whose mature form carries an N-terminal glycine? Return protein URI and AA sequence",
        "question": "How do I filter for reviewed mouse proteins which carry an N-terminal glycine? Return protein URI and AA sequence",
        "endpoint": "https://sparql.uniprot.org/sparql/",
        "query": """PREFIX up: <http://purl.uniprot.org/core/>
    PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT ?protein ?sequence
    WHERE
    {
        ?protein a up:Protein ;
            up:organism taxon:10090 ;  # Taxonomy ID for Mus musculus (Mouse)
            up:reviewed true ;
            up:sequence ?isoform .
        ?isoform rdf:value ?sequence .
        # Ensure the N-terminal amino acid is Glycine (G)
        FILTER (STRSTARTS(?sequence, "G"))
    }""",
    },
    {
        "question": "How could I download a table that only includes the Rhea reactions for which there is experimental evidence? Return only the rhea URI",
        "endpoint": "https://sparql.uniprot.org/sparql/",
        "query": """PREFIX up: <http://purl.uniprot.org/core/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT DISTINCT ?rhea
    WHERE {
    # ECO 269 is experimental evidence
    BIND (<http://purl.obolibrary.org/obo/ECO_0000269> as ?evidence)
    #GRAPH <http://sparql.uniprot.org/uniprot> {
        ?protein up:reviewed true ;
        up:annotation ?a ;
        up:attribution ?attribution  .

        ?a a up:Catalytic_Activity_Annotation ;
        up:catalyticActivity ?ca .
        ?ca up:catalyzedReaction ?rhea .

        [] rdf:subject ?a ;
        rdf:predicate up:catalyticActivity ;
        rdf:object ?ca ;
        up:attribution ?attribution .

        ?attribution up:evidence ?evidence .
    #}
    }""",
    },
    {
        "question": "Which human proteins are enzymes catalyzing a reaction involving sterols? Return the protein, sterol and reaction URI",
        "endpoint": "https://sparql.uniprot.org/sparql/",
        "query": """PREFIX up: <http://purl.uniprot.org/core/>
    PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
    PREFIX rh: <http://rdf.rhea-db.org/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>
    SELECT DISTINCT ?protein ?sterol ?reaction
    WHERE {
    SERVICE <https://sparql.rhea-db.org/sparql> {
        ?reaction rdfs:subClassOf rh:Reaction .
        ?reaction rh:side/rh:contains/rh:compound ?compound .
        ?compound rh:chebi ?sterol .
        ?sterol rdfs:subClassOf* CHEBI:15889 .
    }
    ?protein a up:Protein ;
        up:organism taxon:9606 ;
        up:annotation/up:catalyticActivity/up:catalyzedReaction ?reaction .
    }""",
    },
    {
        # "question": "Which are the human proteins associated with cancer (which have cancer in their disease label)? Return the unique disease label (?diseaseLabel), and HGNC symbol (?hgncSymbol)",
        "question": "Which are the human proteins associated with cancer? Return distinct ?diseaseLabel and ?hgncSymbol",
        "endpoint": "https://sparql.uniprot.org/sparql/",
        "query": """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX up:<http://purl.uniprot.org/core/>
    PREFIX taxon:<http://purl.uniprot.org/taxonomy/>
    PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
    PREFIX orth:<http://purl.org/net/orth#>
    PREFIX dcterms:<http://purl.org/dc/terms/>
    PREFIX obo:<http://purl.obolibrary.org/obo/>
    PREFIX lscr:<http://purl.org/lscr#>
    PREFIX genex:<http://purl.org/genex#>
    PREFIX sio: <http://semanticscience.org/resource/>
    SELECT DISTINCT ?diseaseLabel ?hgncSymbol
    WHERE {
        ?humanProtein a up:Protein ;
            # up:organism/up:scientificName 'Homo sapiens' ;
            up:organism taxon:9606 ;
            up:annotation ?annotation ;
            rdfs:seeAlso ?hgnc .
        ?hgnc up:database <http://purl.uniprot.org/database/HGNC> ;
            rdfs:comment ?hgncSymbol .
        ?annotation a up:Disease_Annotation ;
            up:disease ?disease .
        ?disease skos:prefLabel ?diseaseLabel.
        FILTER CONTAINS (LCASE(?diseaseLabel), "cancer")
    }""",
    },
    {
        "question": "In bgee how can I retrieve the confidence level and false discovery rate of a gene expression? Return distinct ?gene, ?confidence and ?fdr, limit to 10",
        "endpoint": "https://www.bgee.org/sparql/",
        "query": """PREFIX genex: <http://purl.org/genex#>
    PREFIX bgee: <http://bgee.org/#>
    PREFIX obo: <http://purl.obolibrary.org/obo/>
    SELECT DISTINCT ?gene ?confidence ?fdr
    WHERE {
        ?expression a genex:Expression ;
            genex:hasConfidenceLevel ?confidence ;
            genex:hasFDRpvalue ?fdr ;
            genex:hasSequenceUnit ?gene .
    } LIMIT 10""",
    },
    {
        # There are no example with xrefEnsembl, so the RAG without validation usually fails
        "question": "How can I get the cross-reference to the ensembl protein for the LCT protein in OMA? Return only the distinct ?ensemblURI",
        "endpoint": "https://sparql.omabrowser.org/sparql/",
        "query": """PREFIX orth: <http://purl.org/net/orth#>
    PREFIX lscr: <http://purl.org/lscr#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT DISTINCT ?ensemblURI
    WHERE {
        ?protein a orth:Protein ;
            rdfs:label 'LCT' ;
            lscr:xrefEnsemblProtein ?ensemblURI .
    }""",
    },
    {
        # There are no example with inDataset
        "question": "How can I get the URI of a dataset to which an ortholog cluster belongs in OMA? Return orthologCluster, datasetURI and limit to 20",
        "endpoint": "https://sparql.omabrowser.org/sparql/",
        "query": """PREFIX orth: <http://purl.org/net/orth#>
    SELECT DISTINCT ?orthologCluster ?datasetURI
    WHERE {
        ?orthologCluster a orth:OrthologsCluster ;
            orth:inDataset ?datasetURI .
    } LIMIT 20""",
    },
    {
        "question": "Give me the list of strains associated to the Escherichia coli taxon and their name. Return ?taxon, ?strain, ?name, limit to 20",
        "endpoint": "https://sparql.uniprot.org/sparql/",
        "query": """PREFIX up: <http://purl.uniprot.org/core/>
    PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?taxon ?strain ?name
    WHERE {
        ?taxon a up:Taxon .
        ?taxon rdfs:subClassOf taxon:562 . # Escherichia coli taxon
        ?taxon up:strain ?strain .
        ?strain up:name ?name .
    } LIMIT 20""",
    },
    {
        "question": "Retrieve all proteins involved in pathways involving glycolysis. Return ?proteinURI, ?proteinLabel, ?pathwayLabel, limit to 20",
        "endpoint": "https://sparql.uniprot.org/sparql/",
        "query": """PREFIX up: <http://purl.uniprot.org/core/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT DISTINCT ?proteinURI ?proteinLabel ?pathwayLabel
WHERE {
    ?proteinURI a up:Protein ;
        up:recommendedName/up:fullName ?proteinLabel ;
        up:annotation ?annotation .
    ?annotation a up:Pathway_Annotation ;
        rdfs:seeAlso ?pathway .
    ?pathway rdfs:label ?pathwayLabel .
    FILTER(CONTAINS(LCASE(?pathwayLabel), "glycolysis"))
} LIMIT 20""",
    },
    {
        "question": "What are the orthologs in rat for protein Q9Y2T1 ? Return ?ratProtein ?ratUniProtXref",
        "endpoint": "https://sparql.omabrowser.org/sparql/",
        "query": """PREFIX up: <http://purl.uniprot.org/core/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX lscr: <http://purl.org/lscr#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
SELECT DISTINCT ?ratProtein ?ratUniProtXref
WHERE {
    ?cluster a orth:OrthologsCluster.
    ?cluster orth:hasHomologousMember ?node1.
    ?cluster orth:hasHomologousMember ?node2.
    ?node1 orth:hasHomologousMember* ?humanProtein.
    ?node2 orth:hasHomologousMember* ?ratProtein.
    ?humanProtein a orth:Protein;
        lscr:xrefUniprot <http://purl.uniprot.org/uniprot/Q9Y2T1>.
    ?ratProtein a orth:Protein;
        orth:organism/obo:RO_0002162/up:scientificName 'Rattus norvegicus';
        lscr:xrefUniprot ?ratUniProtXref.
    FILTER(?node1 != ?node2)
}""",
    },
    {
        "question": "Retrieve all proteins that are associated with Alzheimer diseases and where they are known to be located in the cell. Return ?proteinURI, ?diseaseLabel, ?locationInsideCellLabel, ?locationInsideCellUri, limit to 20",
        "endpoint": "https://sparql.uniprot.org/sparql/",
        "query": """PREFIX up: <http://purl.uniprot.org/core/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
SELECT ?proteinURI ?diseaseLabel ?locationInsideCellLabel ?locationInsideCellUri
WHERE {
    ?proteinURI a up:Protein ;
        up:annotation ?diseaseAnnotation , ?subcellAnnotation .
    ?diseaseAnnotation up:disease ?disease .
    ?disease skos:prefLabel ?diseaseLabel .
    FILTER(CONTAINS(LCASE(?diseaseLabel), "alzheimer"))
    ?subcellAnnotation up:locatedIn/up:cellularComponent ?locationInsideCellUri .
    ?locationInsideCellUri skos:prefLabel ?locationInsideCellLabel .
} LIMIT 20""",
    },
    {
        "question": "Retrieve all proteins in OMA that are encoded by the TP53 gene and their mnemonics and evidence types from the UniProt database. Return ?proteinOMA ?speciesLabel  ?mnemonic  ?evidenceType ?uniprotURI",
        "endpoint": "https://sparql.omabrowser.org/sparql/",
        "query": """PREFIX up: <http://purl.uniprot.org/core/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX orth: <http://purl.org/net/orth#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX lscr: <http://purl.org/lscr#>
SELECT DISTINCT ?proteinOMA ?speciesLabel  ?mnemonic  ?evidenceType ?uniprotURI
WHERE {
    ?proteinOMA a orth:Protein ;
        orth:organism/obo:RO_0002162/up:scientificName ?speciesLabel ;
        rdfs:label 'TP53' .
    ?proteinOMA lscr:xrefUniprot ?uniprotURI.
    SERVICE <http://sparql.uniprot.org/sparql> {
        ?uniprotURI up:mnemonic ?mnemonic ;
            up:existence/rdfs:label ?evidenceType .
    }
}
""",
    },

    # New queries to test:
    # Find all proteins linked to arachidonate (CHEBI:32395) and their associated pathways
    # List all enzymes that have been experimentally validated and are involved in DNA repair
    # Find all proteins that have a mutagenesis annotation affecting their active site


    #     {
    #         # Way too slow for some reason
    #         "question": "Retrieve all proteins that are associated with Alzheimer disease. Return ?proteinURI, ?proteinLabel, ?diseaseURI, ?diseaseLabel, limit to 20",
    #         "endpoint": "https://sparql.uniprot.org/sparql/",
    #         "query": """PREFIX up: <http://purl.uniprot.org/core/>
    # PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    # PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    # SELECT ?proteinURI ?proteinLabel ?diseaseURI ?diseaseLabel
    # WHERE {
    #     ?proteinURI a up:Protein ;
    #             up:annotation ?diseaseAnnotation ;
    #             up:recommendedName/up:fullName ?proteinLabel .
    #     ?diseaseAnnotation up:disease ?diseaseURI .
    #     ?diseaseURI skos:prefLabel ?diseaseLabel .
    #     FILTER(CONTAINS(LCASE(?diseaseLabel), "alzheimer"))
    # } LIMIT 10""",
    #     },

    #     {
    #         # Always failing the connection between OMA and Bgee
    #         "question": "For protein Q9Y2T1, what are the orthologs in rats for genes expressed in the brain? Return ?ratProtein ?ratUniProtXref",
    #         "endpoint": "https://sparql.omabrowser.org/sparql/",
    #         "query": """PREFIX up: <http://purl.uniprot.org/core/>
    # PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    # PREFIX orth: <http://purl.org/net/orth#>
    # PREFIX lscr: <http://purl.org/lscr#>
    # PREFIX obo: <http://purl.obolibrary.org/obo/>
    # SELECT DISTINCT ?ratProtein ?ratUniProtXref
    # WHERE {
    #     ?cluster a orth:OrthologsCluster.
    #     ?cluster orth:hasHomologousMember ?node1.
    #     ?cluster orth:hasHomologousMember ?node2.
    #     ?node1 orth:hasHomologousMember* ?humanProtein.
    #     ?node2 orth:hasHomologousMember* ?ratProtein.
    #     ?humanProtein a orth:Protein;
    #         lscr:xrefUniprot <http://purl.uniprot.org/uniprot/Q9Y2T1>.
    #     ?ratProtein a orth:Protein;
    #         orth:organism/obo:RO_0002162/up:scientificName 'Rattus norvegicus';
    #         lscr:xrefUniprot ?ratUniProtXref.
    #     FILTER(?node1 != ?node2)
    # }""",
    #     },

    #     {
    #         # It's making up URIs for photosynthesis instead of filtering on label
    #         "question": "Find all proteins that have a known 3D structure and are involved in photosynthesis. Return ?protein, ?structure, ?keywordLabel, limit to 20",
    #         "endpoint": "https://sparql.uniprot.org/sparql/",
    #         "query": """PREFIX up: <http://purl.uniprot.org/core/>
    # PREFIX keywords: <http://purl.uniprot.org/keywords/>
    # PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    # SELECT DISTINCT ?protein ?structure ?keywordLabel
    # WHERE {
    #     ?protein a up:Protein ;
    #         up:classifiedWith/rdfs:label ?keywordLabel ;
    #         rdfs:seeAlso ?structure .
    #     ?structure up:database ?db .
    #     ?db up:category '3D structure databases' .
    #     FILTER(LCASE(?keywordLabel) = "photosynthesis")
    # } LIMIT 20""",
    #     },

    # {
    #     # Asking for "full name" will cause the RAG to fail, but validation don't catch it yet because VoID don't register the class when we have
    #     # we need to wait for LinkSet to be added to VoID in UniProt
    #     # Why not using "up:Protein up:enzyme ??"... Also up:enzymeClass never shows up in the relevant documents
    #         "question": "How can I get a list of proteins that are enzymes with their enzyme class? Return the ?proteinURI, ?enzymeClassURI, ?enzymeClassName, limit to 20",
    #         "endpoint": "https://sparql.uniprot.org/sparql/",
    #         "query": """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    #     PREFIX up: <http://purl.uniprot.org/core/>
    #     PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    #     SELECT ?proteinURI ?enzymeClassURI ?enzymeClassName
    #     WHERE {
    #        ?proteinURI a up:Protein ;
    #                    up:enzyme ?enzymeClassURI .
    #        ?enzymeClassURI skos:prefLabel ?enzymeClassName .
    #     #   ?protein a up:Protein ;
    #     #            up:annotation ?enzymeAnnotation .
    #     #   ?enzymeAnnotation a up:Catalytic_Activity_Annotation ;
    #     #                     up:catalyticActivity ?enzymeActivity .
    #     #   ?enzymeActivity up:enzymeClass ?enzymeClassURI .
    #     #   ?enzymeClassURI skos:prefLabel ?enzymeClassName .
    #     } LIMIT 20""",
    # },

    # {
    #     "question": "Which are the human genes associated with cancer (which have cancer in their disease label) and their orthologs expressed in the rat brain? Return the disease label, human gene URI, human gene HGNC symbol, ortholog rat gene URI",
    #     "endpoint": "https://sparql.uniprot.org/sparql/",
    #     "query": """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    # PREFIX up:<http://purl.uniprot.org/core/>
    # PREFIX taxon:<http://purl.uniprot.org/taxonomy/>
    # PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
    # PREFIX orth:<http://purl.org/net/orth#>
    # PREFIX dcterms:<http://purl.org/dc/terms/>
    # PREFIX obo:<http://purl.obolibrary.org/obo/>
    # PREFIX lscr:<http://purl.org/lscr#>
    # PREFIX genex:<http://purl.org/genex#>
    # PREFIX sio: <http://semanticscience.org/resource/>
    # SELECT DISTINCT ?diseaseLabel ?humanProtein ?hgncSymbol ?orthologRatProtein ?orthologRatGene
    # WHERE {
    # SERVICE <https://sparql.uniprot.org/sparql> {
    #     SELECT DISTINCT * WHERE {
    #     ?humanProtein a up:Protein ;
    #         up:organism/up:scientificName 'Homo sapiens' ;
    #         up:annotation ?annotation ;
    #         rdfs:seeAlso ?hgnc .
    #     ?hgnc up:database <http://purl.uniprot.org/database/HGNC> ;
    #         rdfs:comment ?hgncSymbol .
    #     ?annotation a up:Disease_Annotation ;
    #             up:disease ?disease .
    #     ?disease skos:prefLabel ?diseaseLabel.
    #     FILTER CONTAINS (?diseaseLabel, "cancer")
    #     }
    # }
    # SERVICE <https://sparql.omabrowser.org/sparql/> {
    #     SELECT ?humanProtein ?orthologRatProtein ?orthologRatGene WHERE {
    #     ?humanProteinOma a orth:Protein ;
    #             lscr:xrefUniprot ?humanProtein .
    #     ?orthologRatProtein a orth:Protein ;
    #         sio:SIO_010079 ?orthologRatGene ;
    #         orth:organism/obo:RO_0002162/up:scientificName 'Rattus norvegicus' .
    #     ?cluster a orth:OrthologsCluster .
    #     ?cluster orth:hasHomologousMember ?node1 .
    #     ?cluster orth:hasHomologousMember ?node2 .
    #     ?node1 orth:hasHomologousMember* ?humanProteinOma .
    #     ?node2 orth:hasHomologousMember* ?orthologRatProtein .
    #     FILTER(?node1 != ?node2)
    #     }
    # }
    # SERVICE <https://www.bgee.org/sparql/> {
    #     ?orthologRatGene genex:isExpressedIn ?anatEntity ;
    #         orth:organism ?ratOrganism .
    #     ?anatEntity rdfs:label 'brain' .
    #     ?ratOrganism obo:RO_0002162 taxon:10116 .
    # }
    # }""",
    # },
]


def result_sets_are_same(gen_set, ref_set) -> bool:
    gen_set, ref_set = list(gen_set), list(ref_set)
    for item in gen_set:
        if item not in ref_set:
            print(f"Missing from reference: {item}")
            return False
    for item in ref_set:
        if item not in gen_set:
            print(f"Missing from generated: {item}")
            return False
    return True


# QLEVER_UNIPROT = "https://qlever.cs.uni-freiburg.de/api/uniprot"

# Price per M tokens, open source models based on fireworks.io pricing
# https://openai.com/api/pricing/
# https://fireworks.ai/pricing
models = {
    "gpt-4o-mini": {
        "id": "gpt-4o-mini",
        "price_input": 0.15,
        "price_output": 0.6,
    },
    "Llama3.1 8B": {
        "id": "hf:meta-llama/Meta-Llama-3.1-8B-Instruct",
        "price_input": 0.2,
        "price_output": 0.2,
    },
    "Mixtral 8x22B": {
        "id": "hf:mistralai/Mixtral-8x22B-Instruct-v0.1",
        "price_input": 1.20,
        "price_output": 1.20,
    },
    "gpt-4o": {
        "id": "gpt-4o",
        "price_input": 5,
        "price_output": 15,
    },
}


def answer_no_rag(question: str, model: str):
    client = get_llm_client(model)
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": settings.system_prompt},
            {"role": "user", "content": question},
        ],
        stream=False,
        temperature=0,
        #   response_format={ "type": "json_object" },
    )
    # response.choices[0].message.content
    return response.to_dict()


def answer_rag_without_validation(question: str, model: str):
    response = requests.post(
        "http://localhost:8000/chat",
        json={
            "model": model,
            "messages": [{"role": "user", "content": question}],
            "stream": False,
            "api_key": expasy_api_key,
            "validate_output": False,
        },
        timeout=60,
    )
    return response.json()


def answer_rag_with_validation(question: str, model: str):
    response = requests.post(
        "http://localhost:8000/chat",
        json={
            "model": model,
            "messages": [{"role": "user", "content": question}],
            "stream": False,
            "api_key": expasy_api_key,
        },
        timeout=60,
    )
    return response.json()


# list_of_approaches = [
#     answer_no_rag,
#     answer_rag_without_validation,
#     answer_rag_with_validation,
# ]

list_of_approaches = {
    "No RAG": answer_no_rag,
    "RAG without validation": answer_rag_without_validation,
    "RAG with validation": answer_rag_with_validation,
}

results_data = {
    "Model": [],
    "RAG Approach": [],
    "Success": [],
    "Different Results": [],
    "No Results": [],
    "Errors": [],
    "Price": [],
    # 'Precision': [],
    # "Recall": [],
    "F1": [],
}

number_of_tries = 2

print(f"🧪 Testing {len(example_queries)} queries")

for model_label, model in models.items():
    print(f"\n🧠 Testing model: {model_label}\n")
    res = defaultdict(dict)
    # e.g. res["No RAG"]["success"] += 1
    for approach in list_of_approaches:
        res[approach] = defaultdict(int)

    for _i, test_query in enumerate(example_queries):
        # Execute the reference query
        res_from_ref = query_sparql(test_query["query"], test_query["endpoint"], timeout=200)["results"]["bindings"]
        # res_from_ref = query_sparql(test_query["query"], QLEVER_UNIPROT)["results"]["bindings"]

        for approach, approach_func in list_of_approaches.items():
            for t in range(number_of_tries):
                response = approach_func(test_query["question"], model["id"])
                chat_resp_md = response["choices"][0]["message"]["content"]
                res[approach]["input_tokens"] += response["usage"]["prompt_tokens"]
                res[approach]["output_tokens"] += response["usage"]["completion_tokens"]
                # print(response["usage"])
                # print(chat_resp_md)
                try:
                    generated_sparql = extract_sparql_queries(chat_resp_md)[-1]
                    if generated_sparql["query"].strip() == test_query["query"].strip():
                        print(f"✅ {t+1}/{number_of_tries} {test_query['question']}. EXACT MATCH")
                        res[approach]["success"] += 1
                        continue

                    # Execute the generated query
                    res_from_generated = query_sparql(
                        generated_sparql["query"], generated_sparql["endpoint_url"], timeout=200
                    )["results"]["bindings"]
                    # res_from_generated = query_sparql(generated_sparql["query"], QLEVER_UNIPROT)["results"]["bindings"]

                    if not result_sets_are_same(res_from_generated, res_from_ref):
                        if len(res_from_generated) == 0:
                            res[approach]["no_results"] += 1
                        else:
                            res[approach]["different_results"] += 1
                        raise Exception(
                            f"\nResults mismatch. Ref: {len(res_from_ref)} != gen: {len(res_from_generated)}\n"
                        )
                    else:
                        print(f"✅ {t+1}/{number_of_tries} {test_query['question']} = {len(res_from_generated)}")
                        res[approach]["success"] += 1

                except Exception as e:
                    res[approach]["fail"] += 1
                    if approach != "No RAG":
                        fail_emoji = "❌" if approach == "RAG with validation" else "⛔️"
                        print(f"{fail_emoji} {t+1}/{number_of_tries} {test_query['question']}\n{e}")
                        # print(generated_sparql["query"])
                        # print("Correct query:")
                        # print(test_query["query"])
                        # print("")

        for approach in list_of_approaches:
            print(
                f"🎯 {approach} - Success: {res[approach]['success']}, Different results: {res[approach]['different_results']}, No results: {res[approach]['no_results']}, Error: {res[approach]['fail']-res[approach]['no_results']-res[approach]['different_results']}"
            )

    for approach, result_row in res.items():
        mean_price = (
            (result_row["input_tokens"] * model["price_input"] / 1000000)
            + (result_row["output_tokens"] * model["price_output"] / 1000000)
        ) / (len(example_queries) * number_of_tries)
        precision = result_row["success"] / (result_row["success"] + result_row["fail"])
        recall = result_row["success"] / (result_row["success"] + result_row["fail"] - result_row["different_results"])
        results_data["Model"].append(model_label)
        results_data["RAG Approach"].append(approach)
        results_data["Success"].append(result_row["success"])
        results_data["Different Results"].append(result_row["different_results"])
        results_data["No Results"].append(result_row["no_results"])
        results_data["Errors"].append(result_row["fail"] - result_row["no_results"] - result_row["different_results"])
        results_data["Price"].append(round(mean_price, 5))
        # results_data['Precision'].append(precision)
        # results_data['Recall'].append(recall)
        results_data["F1"].append(2 * (precision * recall) / (precision + recall))


df = pd.DataFrame(results_data)
print(df)
print("\n\n")
print(df.to_csv("tests_results.csv", index=False))

# Output Latex table
latex_str = ""
prev_model = models.keys()[0]
for index, row in df.iterrows():
    row_str = " & ".join([str(item) for item in row])  # Join all values in the row with " & "
    row_str += " \\\\"
    if row["Model"] != prev_model:
        latex_str += "\\midrule\n"
        prev_model = row["Model"]
    latex_str += row_str + "\n"
with open("tests_results_latex.txt", "w") as f:
    f.write(latex_str)

🧪 Testing 13 queries

🧠 Testing model: gpt-4o-mini

Missing from generated: {'protein': {'type': 'uri', 'value': 'http://purl.uniprot.org/uniprot/H0Y4E4'}}
⛔️ 1/2 What is the accession number in uniprot of the human gene LCT? Return only unique protein URIs

Results mismatch. Ref: 4 != gen: 0

Missing from generated: {'protein': {'type': 'uri', 'value': 'http://purl.uniprot.org/uniprot/H0Y4E4'}}
⛔️ 2/2 What is the accession number in uniprot of the human gene LCT? Return only unique protein URIs

Results mismatch. Ref: 4 != gen: 0

Missing from generated: {'protein': {'type': 'uri', 'value': 'http://purl.uniprot.org/uniprot/H0Y4E4'}}
❌ 1/2 What is the accession number in uniprot of the human gene LCT? Return only unique protein URIs

Results mismatch. Ref: 4 != gen: 0

Missing from generated: {'protein': {'type': 'uri', 'value': 'http://purl.uniprot.org/uniprot/H0Y4E4'}}
❌ 2/2 What is the accession number in uniprot of the human gene LCT? Return only unique protein URIs

Results mismat